In [4]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [5]:
df = pd.read_csv('export_TweetSentBR.csv')
df

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [6]:
df.drop('id_twitter', inplace = True, axis = 1)
df.drop('id', inplace = True, axis = 1)
df.head()

,text,sentiment
0,Que coisa linda! O Programa #encontro estava m...,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,Tô passada com esse cara.... quanta merda pode...,-1


In [7]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [8]:
df.drop_duplicates() #deletando também duplicatas, se existirem
df.shape

(11570, 2)

In [9]:
df['sentiment'].value_counts()

1     5217
-1    3388
0     2928
-       37
Name: sentiment, dtype: int64

In [12]:
df_remove = df.loc[(df['sentiment'] == '-')] 
df = df.drop(df_remove.index)
df['sentiment'].value_counts()

1     5217
-1    3388
0     2928
Name: sentiment, dtype: int64

## Pré-processamento do texto

In [14]:
import re

print (df['text'][1])
texto = df['text'][1]

res = re.sub("@[^ ]+", "", texto)

print(res)

Por mais #Encontro com as Irmãs Galvão, adorei elas.
Por mais #Encontro com as Irmãs Galvão, adorei elas.


In [15]:
def user_clean(text):
    res = re.sub("@[^ ]+", "", text)
    return res

for i in df.index:
    texto = df['text'][i]
    df['text'][i] = user_clean(texto)

df

,text,sentiment
0,Que coisa linda! O Programa #encontro estava m...,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,Mr. CATRA lançando sua nova música PPK CHORA ...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...
11565,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,#MaisVoce está linda,1
11568,"Que orgulho de ti, #Encontro",1


In [16]:
import unicodedata

def f_clean(df, text):
    df['text'] = df['text'].replace(regex='[!/,.-]',value='').apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii','ignore').decode("utf-8"))
    return df
                                                                    
f_clean(df, df['text'])

,text,sentiment
0,Que coisa linda O Programa #encontro estava mo...,1
1,Por mais #Encontro com as Irmas Galvao adorei ...,1
2,Mr CATRA lancando sua nova musica PPK CHORA n...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,To passada com esse cara quanta merda pode sai...,-1
...,...,...
11565,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,MALUCO Uma coisa que eu nao tenho coragem e es...,-1
11567,#MaisVoce esta linda,1
11568,Que orgulho de ti #Encontro,1


In [17]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def f_stopwords(df, text):
    stop_words = stopwords.words('portuguese')
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df

f_stopwords(df, df['text'])

[nltk_data] Downloading package stopwords to C:\Users\Letícia
[nltk_data]     Sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,sentiment
0,Que coisa linda O Programa #encontro mostrando...,1
1,Por #Encontro Irmas Galvao adorei,1
2,Mr CATRA lancando nova musica PPK CHORA k k k ...,1
3,viu lutadora modela barbuda tatuada? #MasterCh...,0
4,To passada cara quanta merda pode sair boca al...,-1
...,...,...
11565,ja to aqui pronto pro #MasterChefBR ainda falt...,-1
11566,MALUCO Uma coisa nao coragem lance parto norma...,-1
11567,#MaisVoce linda,1
11568,Que orgulho ti #Encontro,1
